# Generador de Contenido Educativo basado en LLM

## Descargamos e importamos las librerías necesarias

In [ ]:
!pip install backoff
!pip install python-docx
!pip install PyPDF2
!pip install nltk
!pip install markdown-it-py python-docx  
!pip install streamlit
!pip install ipywidgets

In [1]:
import os
import google.generativeai as genai
import time
import json
import logging
from typing import Dict, List, Any, Optional, Union
import backoff
import re

# Bibliotecas para procesamiento de documentos
import PyPDF2
import docx

# Bibliotecas para generación de texto
import pypandoc
import markdown
from markdown_it import MarkdownIt
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

#Bibliotecas para procesamiento de texto

import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Bibliotecas para visualización
from IPython.display import display, HTML, FileLink, clear_output
import ipywidgets as widgets


### Comenzamos inicializando la configuración 


In [ ]:
class Config:
  def __init__(self):
    # Configuración para el LLM y las plantillas
    self.gemini_model = 'gemini-2.0-flash-001'  # Modelo de Google Gemini
    self.llm_tokens_per_minute = 50000  # Límite de tokens por minuto
    self.llm_max_tokens_per_request = 4000  # Máximo tokens por solicitud
    self.google_api_key = os.environ['GOOGLE_API_KEY']
    self.prompt_templates_path = "prompts.json"  # Archivo JSON con las plantillas de prompt
    # Parámetros para la evaluación (pueden extenderse)
    self.readability_threshold = 60.0

Creamos el rate limit para evitar que se bloquee la ejecución del código
y definimos la clase para el manejo de la API de Gemini



In [ ]:

class RateLimiter:
    """Implementa limitación de tasa para llamadas a la API."""

    def __init__(self, tokens_per_minute: int, max_tokens_per_request: int):
        self.tokens_per_minute = tokens_per_minute
        self.max_tokens_per_request = max_tokens_per_request
        self.tokens_used_in_minute = 0
        self.last_reset = time.time()

    def wait_if_needed(self, tokens: int) -> None:
        """Espera si es necesario para respetar los límites de tasa."""
        # Verificar si ha pasado un minuto desde el último reset
        now = time.time()
        if now - self.last_reset >= 60:
            self.tokens_used_in_minute = 0
            self.last_reset = now

        # Verificar si estamos por encima del límite de tokens por minuto
        if self.tokens_used_in_minute + tokens > self.tokens_per_minute:
            # Calcular el tiempo que tenemos que esperar
            time_to_wait = 60 - (now - self.last_reset)
            if time_to_wait > 0:
                time.sleep(time_to_wait)
                self.tokens_used_in_minute = tokens
                self.last_reset = time.time()
        else:
            self.tokens_used_in_minute += tokens

class LLMEngine:
    """Gestiona la interacción con Google Gemini API."""

    def __init__(self, config: Config):
        """Inicializa el motor LLM con la configuración especificada."""
        self.config = config
        self.logger = logging.getLogger("educational_agent.llm_engine")

        # Configurar Gemini API
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        self.model = genai.GenerativeModel(self.config.gemini_model)

        # Configurar el limitador de tasa
        self.rate_limiter = RateLimiter(
            tokens_per_minute=self.config.llm_tokens_per_minute,
            max_tokens_per_request=self.config.llm_max_tokens_per_request
        )

        # Cargar las plantillas de prompts
        self.prompt_templates = self._load_prompt_templates()

    def _load_prompt_templates(self) -> Dict[str, str]:
        """Carga las plantillas de prompts desde un archivo."""
        try:
            with open(self.config.prompt_templates_path, 'r') as f:
                return json.load(f)
        except Exception as e:
            self.logger.warning(f"No se pudieron cargar las plantillas: {str(e)}")
            # Plantillas por defecto
            return {
                "lecture_notes": """Genera notas de clase detalladas para el tema: {topic_title}.
                                  Incluye los siguientes subtemas: {subtopics}.
                                  Las notas deben incluir definiciones, explicaciones claras, ejemplos y casos de aplicación.""",

                "practice_problems": """Crea problemas de práctica con soluciones paso a paso para el tema: {topic_title}.
                                      Los problemas deben cubrir: {subtopics}.
                                      Incluye problemas de distintos niveles de dificultad.""",

                "discussion_questions": """Genera preguntas para discusión sobre el tema: {topic_title}, considerando: {subtopics}.
                                        Las preguntas deben promover el pensamiento crítico y el análisis profundo.""",

                "learning_objectives": """Crea objetivos de aprendizaje específicos y medibles para el tema: {topic_title}.
                                       Considera los siguientes subtemas: {subtopics}.
                                       Usa verbos de la taxonomía de Bloom apropiados.""",

                "suggested_resources": """Sugiere recursos de aprendizaje adicionales para el tema: {topic_title}.
                                       Incluye libros, artículos, videos, cursos en línea y otros materiales relevantes para: {subtopics}."""
            }

    @backoff.on_exception(
        backoff.expo,
        Exception,
        max_tries=3,
        jitter=backoff.full_jitter
    )
    def generate_content(self, prompt: str, max_tokens: int = 1000) -> str:
        """
        Genera contenido llamando a la API de Google Gemini.

        Args:
            prompt: Instrucción para el modelo
            max_tokens: Máximo número de tokens a generar

        Returns:
            Texto generado por el LLM
        """
        self.logger.debug(f"Generando contenido con prompt: {prompt[:100]}...")

        # Estimar tokens en el prompt (aproximado)
        prompt_tokens = len(prompt.split())
        response_tokens = max_tokens
        total_tokens = prompt_tokens + response_tokens

        # Esperar si es necesario para respetar límites de tasa
        self.rate_limiter.wait_if_needed(total_tokens)

        try:
            # Agregar un sistema de instrucciones
            system_instruction = "Eres un asistente educativo experto en crear materiales didácticos de alta calidad para cursos universitarios. Tu tarea es generar contenido educativo claro, preciso y bien estructurado."

            # Configurar generación con Gemini
            generation_config = {
                "max_output_tokens": max_tokens,
                "temperature": 0.7,
                "top_p": 0.9,
                "top_k": 40
            }

            # Llamar a la API de Gemini
            response = self.model.generate_content(
                contents=[
                    {"role": "user", "parts": [{"text": system_instruction + "\n\n" + prompt}]}
                ],
                generation_config=generation_config
            )

            # Extraer el texto generado
            generated_text = response.text

            return generated_text

        except Exception as e:
            self.logger.error(f"Error al generar contenido con Gemini: {str(e)}")
            raise

Vamos a crear la clase que nos permitirá crear el contenido educativo, tales como:

- Notas de clase que explican en detalle cada tema.
- Problemas de práctica con soluciones paso a paso.
- Preguntas para discusión que estimulan el pensamiento crítico.
- Objetivos de aprendizaje claros y medibles.
- Recursos adicionales sugeridos que complementan el estudio.


In [4]:

# Módulo para generar los diferentes tipos de contenido educativo
class ContentGenerator:
    """Genera contenido educativo utilizando un motor LLM."""

    def __init__(self, llm_engine: LLMEngine, config: Config):
        """Inicializa el generador de contenido."""
        self.llm_engine = llm_engine
        self.config = config
        self.logger = logging.getLogger("educational_agent.content_generator")

    def generate_all_materials(self, syllabus_data: Dict[str, Any]) -> Dict[str, Dict[str, str]]:
        """
        Genera todos los materiales didácticos para cada tema del programa.

        Args:
            syllabus_data: Datos estructurados del programa de curso

        Returns:
            Diccionario con todos los materiales generados por tema
        """
        all_content = {}

        # Obtener información del curso para contexto
        course_context = {
            "course_title": syllabus_data["course_title"],
            "course_code": syllabus_data["course_code"],
            "course_description": syllabus_data["description"],
            "course_objectives": syllabus_data["objectives"]
        }

        # Generar contenido para cada tema
        for topic in syllabus_data["topics"]:
            topic_id = topic["id"]
            self.logger.info(f"Generando contenido para el tema {topic_id}: {topic['title']}")

            # Generar los diferentes tipos de contenido
            topic_content = {
                "lecture_notes": self.generate_lecture_notes(topic, course_context),
                "practice_problems": self.generate_practice_problems(topic, course_context),
                "discussion_questions": self.generate_discussion_questions(topic, course_context),
                "learning_objectives": self.generate_learning_objectives(topic, course_context),
                "suggested_resources": self.generate_suggested_resources(topic, course_context)
            }

            all_content[topic_id] = topic_content

        return all_content

    def generate_lecture_notes(self, topic: Dict[str, Any], course_context: Dict[str, Any]) -> str:
        """Genera notas de clase para un tema específico."""
        self.logger.info(f"Generando notas de clase para: {topic['title']}")

        # Construir el prompt usando la plantilla
        prompt_template = self.llm_engine.prompt_templates["lecture_notes"]
        subtopics_text = ", ".join(topic["subtopics"])

        prompt = prompt_template.format(
            topic_title=topic["title"],
            subtopics=subtopics_text
        )

        # Añadir contexto del curso
        course_context_text = f"""
        Información del curso:
        - Título: {course_context['course_title']}
        - Código: {course_context['course_code']}
        - Descripción: {course_context['course_description']}

        Genera notas de clase completas y detalladas que cubran el tema a profundidad.
        Usa un estilo académico pero claro, incluyendo definiciones precisas, ejemplos prácticos,
        y casos de aplicación relevantes. Organiza el contenido de manera lógica y estructurada.
        """

        final_prompt = course_context_text + "\n\n" + prompt

        # Generar el contenido con un límite mayor de tokens
        content = self.llm_engine.generate_content(final_prompt, max_tokens=4000)

        return content

    def generate_practice_problems(self, topic: Dict[str, Any], course_context: Dict[str, Any]) -> str:
        """Genera problemas de práctica para un tema específico."""
        self.logger.info(f"Generando problemas de práctica para: {topic['title']}")

        # Construir el prompt usando la plantilla
        prompt_template = self.llm_engine.prompt_templates["practice_problems"]
        subtopics_text = ", ".join(topic["subtopics"])

        prompt = prompt_template.format(
            topic_title=topic["title"],
            subtopics=subtopics_text
        )

        # Añadir contexto y instrucciones específicas
        additional_context = f"""
        Para el curso: {course_context['course_title']} ({course_context['course_code']})

        Genera un conjunto de problemas de práctica que cubran todos los aspectos importantes del tema.
        Incluye:
        1. Al menos 5 problemas de diferentes niveles de dificultad (básico, intermedio, avanzado)
        2. Cada problema debe tener una solución paso a paso detallada
        3. Los problemas deben ser relevantes para el contexto del curso
        4. Incluye una mezcla de problemas conceptuales y aplicados
        """

        final_prompt = additional_context + "\n\n" + prompt

        # Generar el contenido
        content = self.llm_engine.generate_content(final_prompt, max_tokens=3000)

        return content

    def generate_discussion_questions(self, topic: Dict[str, Any], course_context: Dict[str, Any]) -> str:
        """Genera preguntas para discusión sobre un tema específico."""
        self.logger.info(f"Generando preguntas para discusión para: {topic['title']}")

        # Construir el prompt usando la plantilla
        prompt_template = self.llm_engine.prompt_templates["discussion_questions"]
        subtopics_text = ", ".join(topic["subtopics"])

        prompt = prompt_template.format(
            topic_title=topic["title"],
            subtopics=subtopics_text
        )

        # Añadir contexto y instrucciones específicas
        additional_context = f"""
        Para el curso: {course_context['course_title']}

        Genera un conjunto de preguntas para discusión que:
        1. Promuevan el pensamiento crítico y reflexivo
        2. Estimulen el debate y el intercambio de ideas
        3. Conecten el tema con problemas actuales o aplicaciones reales
        4. Exploren implicaciones éticas o sociales cuando sea apropiado
        5. Fomenten la conexión entre este tema y otros del curso

        Para cada pregunta, incluye una breve nota para el instructor sobre puntos clave a considerar.
        """

        final_prompt = additional_context + "\n\n" + prompt

        # Generar el contenido
        content = self.llm_engine.generate_content(final_prompt, max_tokens=2000)

        return content

    def generate_learning_objectives(self, topic: Dict[str, Any], course_context: Dict[str, Any]) -> str:
        """Genera objetivos de aprendizaje para un tema específico."""
        self.logger.info(f"Generando objetivos de aprendizaje para: {topic['title']}")

        # Construir el prompt usando la plantilla
        prompt_template = self.llm_engine.prompt_templates["learning_objectives"]
        subtopics_text = ", ".join(topic["subtopics"])

        prompt = prompt_template.format(
            topic_title=topic["title"],
            subtopics=subtopics_text
        )

        # Añadir contexto y instrucciones específicas
        additional_context = f"""
        Para el curso: {course_context['course_title']}

        Genera objetivos de aprendizaje que:
        1. Sean específicos, medibles, alcanzables, relevantes y con tiempo definido (SMART)
        2. Utilicen verbos de acción de la taxonomía de Bloom apropiados para el nivel universitario
        3. Cubran diferentes niveles cognitivos (conocimiento, comprensión, aplicación, análisis, evaluación, creación)
        4. Se alineen con los objetivos generales del curso
        5. Sean claros y comprensibles para los estudiantes

        Los objetivos generales del curso son:
        {', '.join(course_context['course_objectives'])}
        """

        final_prompt = additional_context + "\n\n" + prompt

        # Generar el contenido
        content = self.llm_engine.generate_content(final_prompt, max_tokens=1500)

        return content

    def generate_suggested_resources(self, topic: Dict[str, Any], course_context: Dict[str, Any]) -> str:
        """Genera recursos sugeridos para un tema específico."""
        self.logger.info(f"Generando recursos sugeridos para: {topic['title']}")

        # Construir el prompt usando la plantilla
        prompt_template = self.llm_engine.prompt_templates["suggested_resources"]
        subtopics_text = ", ".join(topic["subtopics"])

        prompt = prompt_template.format(
            topic_title=topic["title"],
            subtopics=subtopics_text
        )

        # Añadir contexto y instrucciones específicas
        additional_context = f"""
        Para el curso: {course_context['course_title']}

        Genera una lista de recursos de aprendizaje que incluya:
        1. Libros de texto principales y complementarios (con autores y años)
        2. Artículos académicos relevantes y actualizados
        3. Recursos en línea de calidad (cursos, tutoriales, videos)
        4. Herramientas o software relevantes cuando sea aplicable
        5. Recursos para diferentes niveles de conocimiento previo

        Para cada recurso, incluye una breve descripción de su relevancia y utilidad.
        """

        final_prompt = additional_context + "\n\n" + prompt

        # Generar el contenido
        content = self.llm_engine.generate_content(final_prompt, max_tokens=2000)

        return content

Vamos a crear la clase que nos permitirá obtener el archivo con la estructura del curso y extraer la informacion importante para que el agente pueda generar el contenido educativo.


In [5]:

# Módulo para procesar y extraer información de documentos (PDF, DOCX, TXT)

class DocumentProcessor:
    """Clase para procesar y extraer información de programas de cursos."""

    def __init__(self, config: Config):
        """Inicializa el procesador de documentos."""
        self.config = config
        self.logger = logging.getLogger("educational_agent.document_processor")

        # Descargar recursos de NLTK si es necesario
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

    def process_file(self, file_path: str) -> Dict[str, Any]:
        """
        Procesa un archivo de programa de curso y extrae su estructura.

        Args:
            file_path: Ruta al archivo de programa de curso

        Returns:
            Diccionario con la información estructurada del programa
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"No se encontró el archivo: {file_path}")

        # Determinar el tipo de archivo por su extensión
        file_ext = os.path.splitext(file_path)[1].lower()

        # Extraer texto según el tipo de archivo
        if file_ext == '.pdf':
            text = self._extract_text_from_pdf(file_path)
        elif file_ext == '.docx':
            text = self._extract_text_from_docx(file_path)
        elif file_ext == '.txt':
            text = self._extract_text_from_txt(file_path)
        else:
            raise ValueError(f"Formato de archivo no soportado: {file_ext}")

        # Analizar el texto extraído para obtener la estructura del curso
        syllabus_data = self._parse_syllabus(text)

        return syllabus_data

    def _extract_text_from_pdf(self, file_path: str) -> str:
        """Extrae texto de un archivo PDF."""
        self.logger.info(f"Extrayendo texto de PDF: {file_path}")

        text = ""
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    text += page.extract_text() + "\n"

            return text
        except Exception as e:
            self.logger.error(f"Error al extraer texto de PDF: {str(e)}")
            raise

    def _extract_text_from_docx(self, file_path: str) -> str:
        """Extrae texto de un archivo DOCX."""
        self.logger.info(f"Extrayendo texto de DOCX: {file_path}")

        try:
            doc = docx.Document(file_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            return text
        except Exception as e:
            self.logger.error(f"Error al extraer texto de DOCX: {str(e)}")
            raise

    def _extract_text_from_txt(self, file_path: str) -> str:
        """Extrae texto de un archivo de texto plano."""
        self.logger.info(f"Extrayendo texto de TXT: {file_path}")

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except UnicodeDecodeError:
            # Intentar con otra codificación si utf-8 falla
            with open(file_path, 'r', encoding='latin-1') as file:
                return file.read()
        except Exception as e:
            self.logger.error(f"Error al extraer texto de TXT: {str(e)}")
            raise

    def _parse_syllabus(self, text: str) -> Dict[str, Any]:
        """
        Analiza el texto del programa para extraer su estructura.

        Args:
            text: Texto completo del programa de curso

        Returns:
            Diccionario con la información estructurada del programa
        """
        self.logger.info("Analizando estructura del programa de curso")

        # Inicializar estructura de datos del programa
        syllabus_data = {
            "course_title": "",
            "course_code": "",
            "instructor": "",
            "description": "",
            "objectives": [],
            "topics": [],
            "evaluation_methods": [],
            "bibliography": []
        }

        # Dividir en secciones
        sections = self._split_into_sections(text)

        # Extraer información básica del curso
        syllabus_data["course_title"] = self._extract_course_title(sections)
        syllabus_data["course_code"] = self._extract_course_code(sections)
        syllabus_data["instructor"] = self._extract_instructor(sections)
        syllabus_data["description"] = self._extract_description(sections)

        # Extraer objetivos del curso
        syllabus_data["objectives"] = self._extract_objectives(sections)

        # Extraer temario (la parte más importante)
        syllabus_data["topics"] = self._extract_topics(sections)

        # Extraer métodos de evaluación
        syllabus_data["evaluation_methods"] = self._extract_evaluation_methods(sections)

        # Extraer bibliografía
        syllabus_data["bibliography"] = self._extract_bibliography(sections)

        return syllabus_data

    def _split_into_sections(self, text: str) -> Dict[str, str]:
        """Divide el texto en secciones basadas en encabezados comunes."""
        # Lista de posibles encabezados de secciones en programas de curso
        section_headers = [
            r"(?:TÍTULO|NOMBRE)\s+DEL\s+CURSO",
            r"(?:CÓDIGO|CLAVE)",
            r"(?:PROFESOR|INSTRUCTOR|DOCENTE)",
            r"(?:DESCRIPCIÓN|DESCRIPCION)",
            r"(?:OBJETIVOS|METAS)",
            r"(?:TEMARIO|CONTENIDO|PROGRAMA|UNIDADES)",
            r"(?:EVALUACIÓN|EVALUACION|CALIFICACIÓN)",
            r"(?:BIBLIOGRAFÍA|BIBLIOGRAFIA|REFERENCIAS)"
        ]

        sections = {}
        current_section = "preamble"
        sections[current_section] = ""

        # Dividir el texto en líneas
        lines = text.split('\n')

        for line in lines:
            # Comprobar si la línea es un encabezado de sección
            is_header = False
            for pattern in section_headers:
                if re.search(pattern, line, re.IGNORECASE):
                    current_section = line.strip()
                    sections[current_section] = ""
                    is_header = True
                    break

            if not is_header:
                sections[current_section] += line + "\n"

        return sections

    def _extract_course_title(self, sections: Dict[str, str]) -> str:
        """Extrae el título del curso de las secciones."""
        for header, content in sections.items():
            if re.search(r"(?:TÍTULO|NOMBRE)\s+DEL\s+CURSO", header, re.IGNORECASE):
                return content.strip()

        # Si no hay sección específica, buscar en el preámbulo
        if "preamble" in sections:
            lines = sections["preamble"].split('\n')
            for line in lines[:5]:  # Buscar en las primeras 5 líneas
                if len(line.strip()) > 0:
                    return line.strip()

        return "No se pudo determinar el título del curso"

    def _extract_course_code(self, sections: Dict[str, str]) -> str:
        """Extrae el código del curso de las secciones."""
        for header, content in sections.items():
            if re.search(r"(?:CÓDIGO|CLAVE)", header, re.IGNORECASE):
                return content.strip()

        # Buscar patrones de código en todo el texto
        for _, content in sections.items():
            # Buscar patrones típicos de códigos de curso (letras y números)
            code_match = re.search(r"\b[A-Z]{2,4}\s*\d{3,4}\b", content)
            if code_match:
                return code_match.group(0).strip()

        return "No se pudo determinar el código del curso"

    def _extract_instructor(self, sections: Dict[str, str]) -> str:
        """Extrae el nombre del instructor del curso."""
        for header, content in sections.items():
            if re.search(r"(?:PROFESOR|INSTRUCTOR|DOCENTE)", header, re.IGNORECASE):
                return content.strip()

        return "No se pudo determinar el instructor del curso"

    def _extract_description(self, sections: Dict[str, str]) -> str:
        """Extrae la descripción del curso."""
        for header, content in sections.items():
            if re.search(r"(?:DESCRIPCIÓN|DESCRIPCION)", header, re.IGNORECASE):
                return content.strip()

        return "No se encontró descripción del curso"

    def _extract_objectives(self, sections: Dict[str, str]) -> List[str]:
        """Extrae los objetivos del curso."""
        objectives = []

        for header, content in sections.items():
            if re.search(r"(?:OBJETIVOS|METAS)", header, re.IGNORECASE):
                lines = content.split('\n')
                for line in lines:
                    line = line.strip()
                    if line and (line.startswith('-') or line.startswith('•') or re.match(r"^\d+\.", line)):
                        objectives.append(line.lstrip('-•0123456789. '))
                    elif len(line) > 20 and not re.match(r"^\s*$", line):
                        # Líneas largas que podrían ser objetivos sin formato de lista
                        sentences = sent_tokenize(line)
                        for sentence in sentences:
                            if len(sentence) > 20:
                                objectives.append(sentence.strip())

        return objectives

    def _extract_topics(self, sections: Dict[str, str]) -> List[Dict[str, Any]]:
        """Extrae el temario del curso."""
        topics = []

        for header, content in sections.items():
            if re.search(r"(?:TEMARIO|CONTENIDO|PROGRAMA|UNIDADES)", header, re.IGNORECASE):
                # Dividir en posibles unidades o temas principales
                unit_pattern = r"(?:Unidad|Tema|Módulo|Capítulo)\s+(\d+|[IVXLCDM]+)[\s:.]+(.+?)(?=(?:Unidad|Tema|Módulo|Capítulo)\s+\d+|$)"
                units = re.finditer(unit_pattern, content, re.IGNORECASE | re.DOTALL)

                for match in units:
                    unit_num = match.group(1)
                    unit_content = match.group(2).strip()

                    # Extraer subtemas
                    subtopics = []
                    lines = unit_content.split('\n')
                    for line in lines:
                        line = line.strip()
                        if line and (line.startswith('-') or line.startswith('•') or re.match(r"^\d+\.\d+", line)):
                            subtopics.append(line.lstrip('-•0123456789. '))

                    # Si no encontramos subtemas con el formato de lista, intentar por oraciones
                    if not subtopics:
                        sentences = sent_tokenize(unit_content)
                        title = sentences[0] if sentences else "Sin título"
                        subtopics = [s.strip() for s in sentences[1:] if len(s.strip()) > 10]
                    else:
                        # El título probablemente sea la primera línea
                        title = lines[0].strip() if lines else "Sin título"
                        # Limpiar el título de numeraciones
                        title = re.sub(r"^\d+\.\s*", "", title)

                    topics.append({
                        "id": unit_num,
                        "title": title,
                        "subtopics": subtopics
                    })

                # Si no se detectaron unidades con el patrón anterior
                if not topics:
                    # Intento más simple por líneas con numeración
                    numbered_lines = re.finditer(r"^\s*(\d+)\.\s*(.+)$", content, re.MULTILINE)
                    current_topic = None

                    for match in numbered_lines:
                        num = match.group(1)
                        text = match.group(2).strip()

                        # Si es un número de un solo dígito, probablemente es un tema principal
                        if len(num) == 1:
                            current_topic = {
                                "id": num,
                                "title": text,
                                "subtopics": []
                            }
                            topics.append(current_topic)
                        # Si tenemos un tema actual y este es un subtema
                        elif current_topic is not None:
                            current_topic["subtopics"].append(text)

                # Si aún no tenemos temas, dividir por líneas no vacías
                if not topics:
                    current_id = 1
                    for line in content.split('\n'):
                        line = line.strip()
                        if line and len(line) > 5:
                            topics.append({
                                "id": str(current_id),
                                "title": line,
                                "subtopics": []
                            })
                            current_id += 1

        return topics

    def _extract_evaluation_methods(self, sections: Dict[str, str]) -> List[Dict[str, Any]]:
        """Extrae los métodos de evaluación del curso."""
        evaluation_methods = []

        for header, content in sections.items():
            if re.search(r"(?:EVALUACIÓN|EVALUACION|CALIFICACIÓN)", header, re.IGNORECASE):
                # Buscar elementos de evaluación y sus porcentajes
                eval_items = re.finditer(r"([^:]+):\s*(\d+)%", content)

                for match in eval_items:
                    item = match.group(1).strip()
                    percentage = int(match.group(2))

                    evaluation_methods.append({
                        "method": item,
                        "percentage": percentage
                    })

                # Si no encontramos con el patrón anterior, buscar líneas con porcentajes
                if not evaluation_methods:
                    for line in content.split('\n'):
                        line = line.strip()
                        if '%' in line:
                            parts = line.split('%')
                            percentage_part = parts[0].strip()
                            # Extraer el último número antes del %
                            percentage_match = re.search(r"(\d+)\s*$", percentage_part)

                            if percentage_match:
                                percentage = int(percentage_match.group(1))
                                # El método es todo antes del número
                                method = re.sub(r"\d+\s*$", "", percentage_part).strip()

                                evaluation_methods.append({
                                    "method": method,
                                    "percentage": percentage
                                })

        return evaluation_methods

    def _extract_bibliography(self, sections: Dict[str, str]) -> List[str]:
        """Extrae la bibliografía del curso."""
        bibliography = []

        for header, content in sections.items():
            if re.search(r"(?:BIBLIOGRAFÍA|BIBLIOGRAFIA|REFERENCIAS)", header, re.IGNORECASE):
                lines = content.split('\n')
                current_entry = ""

                for line in lines:
                    line = line.strip()
                    if not line:
                        if current_entry:
                            bibliography.append(current_entry)
                            current_entry = ""
                    else:
                        if not current_entry and (line.startswith('-') or line.startswith('•') or re.match(r"^\d+\.", line)):
                            current_entry = line.lstrip('-•0123456789. ')
                        elif not current_entry:
                            current_entry = line
                        else:
                            current_entry += " " + line

                # No olvidar la última entrada
                if current_entry:
                    bibliography.append(current_entry)

                # Si no hay entradas con el formato anterior, dividir por líneas no vacías
                if not bibliography:
                    bibliography = [line.strip() for line in lines if line.strip() and len(line.strip()) > 10]

        return bibliography

Vamos a probar la clase para verificar que se obtiene la información necesaria para generar el contenido educativo.

In [20]:
config = Config()
processor = DocumentProcessor(config)

In [21]:
syllabus_data = processor.process_file('PROGRAMA_DE_CURSO.pdf')

In [25]:
# Initialize instances of the required classes
config = Config() #Assuming Config is defined and accessible
llm_engine = LLMEngine(config) #Initialize LLMEngine before EducationalAgent


# Create a ContentGenerator instance, passing the LLMEngine instance
generador = ContentGenerator(llm_engine,config)

No se pudieron cargar las plantillas: [Errno 2] No such file or directory: 'prompts.json'


In [142]:
contenido = generador.generate_all_materials(syllabus_data)

INFO:educational_agent.content_generator:Generando contenido para el tema 1: Introducción a la IA
INFO:educational_agent.content_generator:Generando notas de clase para: Introducción a la IA
INFO:educational_agent.content_generator:Generando problemas de práctica para: Introducción a la IA
INFO:educational_agent.content_generator:Generando preguntas para discusión para: Introducción a la IA
INFO:educational_agent.content_generator:Generando objetivos de aprendizaje para: Introducción a la IA
INFO:educational_agent.content_generator:Generando recursos sugeridos para: Introducción a la IA
INFO:educational_agent.content_generator:Generando contenido para el tema 2: Algoritmos de Búsqueda
INFO:educational_agent.content_generator:Generando notas de clase para: Algoritmos de Búsqueda
INFO:educational_agent.content_generator:Generando problemas de práctica para: Algoritmos de Búsqueda
INFO:educational_agent.content_generator:Generando preguntas para discusión para: Algoritmos de Búsqueda
INFO

In [143]:
contenido

{'1': {'lecture_notes': '## Notas de Clase: Introducción a la Inteligencia Artificial\n\n**Curso:** Introducción a la Inteligencia Artificial\n**Código:** [Dejar espacio para el código del curso]\n**Descripción:** Este curso introductorio cubre los conceptos básicos de la Inteligencia Artificial, incluyendo algoritmos de búsqueda, aprendizaje automático y procesamiento del lenguaje natural. Los estudiantes desarrollarán habilidades prácticas mediante proyectos aplicados.\n\n**Tema:** Introducción a la IA\n\n**Objetivo:** Proporcionar una comprensión fundamental de la historia, los conceptos clave, las arquitecturas y las consideraciones éticas de la Inteligencia Artificial (IA).\n\n**Índice:**\n\n1.  **¿Qué es la Inteligencia Artificial?**\n    *   1.1 Definición y Alcance\n    *   1.2 Tipos de IA: Débil vs. Fuerte\n2.  **Historia de la IA**\n    *   2.1 Los Inicios (1943-1956): El Nacimiento de una Idea\n    *   2.2 Los Años Dorados (1956-1974): Optimismo y Primeros Logros\n    *   2.

Creamos la funcion principal para exportar el contenido educativo en formato pdf.

In [22]:

def dict_to_pdf(materials: dict, output_pdf: str = "materiales_curso.pdf"):
    """
    Convierte un diccionario de materiales a un PDF.
    
    El diccionario se transforma en un string Markdown que luego se convierte a HTML y finalmente a PDF usando pypandoc.
    
    Args:
        materials: Diccionario con la estructura de temas y secciones.
        output_pdf: Nombre del archivo PDF de salida.
    """
    # Construir un string Markdown combinando cada tema y sus secciones
    md_content = ""
    for topic_id, content in materials.items():
        md_content += f"# Tema {topic_id}\n\n"
        for section, text in content.items():
            section_title = section.replace('_', ' ').title()
            md_content += f"## {section_title}\n\n"
            md_content += text + "\n\n"
        md_content += "\n---\n\n"  # separador entre temas

    # Convertir el Markdown a HTML (esto es opcional, ya que pypandoc puede convertir directamente desde Markdown)
    html = markdown.markdown(md_content)
    
    # Opciones adicionales para pypandoc (usa XeLaTeX y una fuente que soporte Unicode)
    extra_args = [
        '--pdf-engine=xelatex',
        '-V', 'mainfont=Times New Roman',
    ]
    
    # Convertir HTML a PDF
    pypandoc.convert_text(html, 'pdf', format='html', outputfile=output_pdf, extra_args=extra_args)
    print(f"✅ Archivo PDF guardado: {output_pdf}")


In [100]:
dict_to_pdf(contenido,"materiales_curso.pdf")

Archivo PDF guardado: materiales_curso.pdf


Como complemento, vamos a crear una función que nos permita exportar el contenido educativo en formato .docx

In [7]:

def dict_to_docx(materials: dict, filename: str = "materiales_curso.docx"):
    """Convierte el diccionario de materiales a PDF conservando formato Markdown"""
    
    # Crear documento Word
    doc = Document()
    
    # Configuración de estilos
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Arial'
    font.size = Pt(11)
    
    # Función para añadir texto con formato
    def add_section(title: str, content: str, level: int = 1):
        # Añadir título
        heading = doc.add_heading(level=level)
        heading_run = heading.add_run(title)
        heading_run.bold = True
        
        # Convertir Markdown a HTML y luego a texto formateado
        md = MarkdownIt()
        tokens = md.parse(content)
        
        # Procesar tokens Markdown
        for token in tokens:
            if token.type == 'inline':
                p = doc.add_paragraph()
                p.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                
                # Manejar diferentes formatos
                if 'strong' in token.map:
                    run = p.add_run(token.content)
                    run.bold = True
                elif 'em' in token.map:
                    run = p.add_run(token.content)
                    run.italic = True
                else:
                    p.add_run(token.content)
                
                # Manejar listas
                if token.type == 'list_item':
                    p.style = 'List Bullet'
                
        doc.add_paragraph()  # Espacio entre secciones

    # Generar contenido
    for topic_id, content in materials.items():
        # Tema principal
        doc.add_heading(f'Tema {topic_id}', level=0)
        
        # Secciones
        for section, text in content.items():
            section_title = section.replace('_', ' ').title()
            add_section(section_title, text, level=1)
            
        doc.add_page_break()

    # Guardar archivo en la ruta especificada
    doc.save(filename)
    print(f"Archivo guardado en: {filename}")

# Ejemplo de uso


# Guardar en la ruta deseada
# output_path = "./materiales_curso.docx"  # Cambia esta ruta
# dict_to_docx(contenido, output_path)

In [8]:

def save_materials_as_text_files(materials: dict, output_dir: str):
    """
    Guarda los materiales generados en archivos de texto.
    
    Cada tema se guarda en una subcarpeta, y para cada tipo de contenido se crea un archivo.
    
    Args:
        materials: Diccionario con los materiales generados.
        output_dir: Directorio base donde se guardarán los archivos.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for topic_id, content in materials.items():
        # Crear una subcarpeta para cada tema, usando el ID o el título (si se prefiere)
        topic_dir = os.path.join(output_dir, f"tema_{topic_id}")
        if not os.path.exists(topic_dir):
            os.makedirs(topic_dir)
        
        # Guardar cada tipo de material en un archivo distinto
        for content_type, text in content.items():
            filename = os.path.join(topic_dir, f"{content_type}.txt")
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(text)
    print(f"Materiales guardados en: {output_dir}")


    

# save_materials_as_text_files(contenido, "materiales_generados")


Para evualar el contenido vamos a crear una clase que analiza la calidad del contenido educativo generado. En esencia, lo que hace es lo siguiente:
1. Evaluación por Tema:

* Para cada tema del curso, el evaluador examina diferentes tipos de contenido (notas de clase, problemas de práctica, preguntas para discusión, objetivos de aprendizaje y recursos sugeridos).
* Cada uno de estos componentes se evalúa mediante funciones específicas que calculan una puntuación basada en varios criterios, como la cobertura de subtemas, la legibilidad (por ejemplo, la longitud promedio de las oraciones), el uso adecuado de terminología del dominio y otros indicadores como la presencia de ejemplos, soluciones o la variedad de recursos.
*Se calcula una puntuación promedio para cada tema a partir de las evaluaciones individuales de cada tipo de contenido.

2. Cálculo de Métricas Globales:

* **Relevancia**: Se mide comparando la cobertura de subtemas esperados en el syllabus con lo que efectivamente se aborda en el contenido. Se calcula la proporción de subtemas que están cubiertos en, por ejemplo, las notas de clase.
+ **Consistencia**: Se evalúa la coherencia del contenido mediante la similitud semántica entre diferentes secciones o temas. Para ello, se representa cada bloque de contenido con vectores TF-IDF y se calcula la similitud de coseno entre ellos. Una mayor similitud indica mayor consistencia en el estilo y terminología.
* **Legibilidad**: Se estima a partir de la longitud promedio de las oraciones. Se asume que oraciones más cortas facilitan la comprensión, por lo que se traduce ese promedio en una puntuación de legibilidad.
* **Uso de Terminología Específica**: Se extraen términos clave del syllabus usando técnicas TF-IDF y luego se verifica la densidad de estos términos en el contenido generado, midiendo qué tan bien se integra la terminología relevante del dominio.

3. Integración de Resultados:

*Las puntuaciones individuales para cada tipo de contenido se promedian para obtener una puntuación global por tema.
*Además, se calculan promedios globales para cada tipo de contenido (por ejemplo, promedios de las notas de clase, problemas de práctica, etc.) y se combinan con las métricas globales (relevancia, consistencia, legibilidad y uso de terminología) para obtener una puntuación global promedio del contenido generado.

En resumen, el evaluador toma el contenido generado, lo compara con la estructura y los subtemas definidos en el syllabus, y aplica varios análisis (por ejemplo, de cobertura de subtemas, similitud semántica, legibilidad y densidad terminológica) para producir una evaluación cuantitativa que refleja la calidad y coherencia del material educativo.

In [9]:

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

class Evaluator:
    """Evalúa la calidad del contenido educativo generado."""
    
    def __init__(self, config):
        self.config = config
        self.logger = logging.getLogger("educational_agent.evaluator")
        
    def evaluate_content(self, generated_content: Dict[str, Dict[str, str]], 
                         syllabus_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Evalúa el contenido generado utilizando diversas métricas:
          - Relevancia (a través de la cobertura de subtemas)
          - Consistencia (usando, por ejemplo, similitud de coseno entre secciones)
          - Legibilidad (a través de la longitud promedio de oraciones)
          - Uso de terminología específica del dominio (extracción TF-IDF)
          
        Args:
            generated_content: Diccionario con materiales generados, organizado por tema y tipo.
            syllabus_data: Datos estructurados del programa del curso.
            
        Returns:
            Diccionario con resultados de evaluación.
        """
        self.logger.info("Iniciando evaluación del contenido generado")
        
        evaluation_results = {
            "topic_scores": {},
            "content_type_scores": {
                "lecture_notes": 0,
                "practice_problems": 0,
                "discussion_questions": 0,
                "learning_objectives": 0,
                "suggested_resources": 0
            },
            "overall_metrics": {
                "relevance_score": 0,
                "consistency_score": 0,
                "readability_score": 0,
                "domain_terminology_score": 0
            },
            "average_score": 0
        }
        
        # Extraer terminología del syllabus para evaluar uso de términos
        course_terminology = self._extract_domain_terminology(syllabus_data)
        
        # Evaluar cada tema
        for topic_id, topic_content in generated_content.items():
            # Buscar la información del tema en el syllabus (ejemplo: usando id)
            topic_info = next((t for t in syllabus_data["topics"] if t["id"] == topic_id), None)
            if not topic_info:
                self.logger.warning(f"No se encontró información para el tema {topic_id} en el syllabus")
                continue
            
            # Evaluar cada tipo de contenido para el tema actual
            lecture = topic_content.get("lecture_notes", "")
            practice = topic_content.get("practice_problems", "")
            discussion = topic_content.get("discussion_questions", "")
            objectives = topic_content.get("learning_objectives", "")
            resources = topic_content.get("suggested_resources", "")
            
            topic_scores = {
                "lecture_notes": self._evaluate_lecture_notes(lecture, topic_info, course_terminology),
                "practice_problems": self._evaluate_practice_problems(practice, topic_info),
                "discussion_questions": self._evaluate_discussion_questions(discussion, topic_info),
                "learning_objectives": self._evaluate_learning_objectives(objectives, topic_info),
                "suggested_resources": self._evaluate_suggested_resources(resources, topic_info)
            }
            
            # Puntuación promedio para el tema
            topic_avg = sum(topic_scores.values()) / len(topic_scores)
            topic_scores["average"] = topic_avg
            
            evaluation_results["topic_scores"][topic_id] = topic_scores
            
            # Acumular para cada tipo de contenido
            for key, score in topic_scores.items():
                if key != "average" and key in evaluation_results["content_type_scores"]:
                    evaluation_results["content_type_scores"][key] += score
        
        # Promediar para cada tipo de contenido
        num_topics = len(generated_content)
        if num_topics > 0:
            for key in evaluation_results["content_type_scores"]:
                evaluation_results["content_type_scores"][key] /= num_topics
        
        # Métricas globales: por ejemplo, podemos usar la similitud entre secciones para consistencia
        overall_consistency = self._evaluate_consistency(generated_content)
        overall_readability = self._calculate_readability(self._join_all_texts(generated_content))
        overall_terminology = self._calculate_terminology_usage(self._join_all_texts(generated_content), course_terminology)
        
        evaluation_results["overall_metrics"]["consistency_score"] = overall_consistency
        evaluation_results["overall_metrics"]["readability_score"] = overall_readability
        evaluation_results["overall_metrics"]["domain_terminology_score"] = overall_terminology
        
        # Relevancia: podemos considerar el promedio de cobertura de subtemas
        overall_relevance = np.mean([
            self._calculate_subtopic_coverage(topic_content.get("lecture_notes", ""), topic_info["subtopics"])
            for topic_id, topic_content in generated_content.items()
            if topic_id in [t["id"] for t in syllabus_data["topics"]]
        ])
        evaluation_results["overall_metrics"]["relevance_score"] = overall_relevance
        
        # Puntuación global promedio
        content_type_avg = np.mean(list(evaluation_results["content_type_scores"].values()))
        overall_metrics_avg = np.mean(list(evaluation_results["overall_metrics"].values()))
        evaluation_results["average_score"] = (content_type_avg + overall_metrics_avg) / 2
        
        self.logger.info(f"Evaluación completada. Puntuación promedio: {evaluation_results['average_score']:.2f}")
        return evaluation_results

    def _join_all_texts(self, generated_content: Dict[str, Dict[str, str]]) -> str:
        """Une todo el texto de todos los temas y secciones para análisis global."""
        texts = []
        for topic_content in generated_content.values():
            for text in topic_content.values():
                texts.append(text)
        return "\n".join(texts)

    def _calculate_readability(self, text: str) -> float:
        """
        Calcula la legibilidad basada en la longitud promedio de las oraciones.
        Se asume que oraciones más cortas son más fáciles de leer.
        """
        sentences = sent_tokenize(text)
        words = word_tokenize(text)
        if not sentences:
            return 0.0
        avg_sentence_length = len(words) / len(sentences)
        if avg_sentence_length <= 15:
            return 1.0
        elif avg_sentence_length >= 30:
            return 0.0
        else:
            return (30 - avg_sentence_length) / 15

    def _extract_domain_terminology(self, syllabus_data: Dict[str, Any]) -> List[str]:
        """
        Extrae términos clave del syllabus usando TF-IDF o simplemente 
        combinando palabras relevantes de la descripción, objetivos, y temario.
        """
        combined_text = syllabus_data.get("description", "")
        for obj in syllabus_data.get("objectives", []):
            combined_text += " " + obj
        for topic in syllabus_data.get("topics", []):
            combined_text += " " + topic.get("title", "")
            for subtopic in topic.get("subtopics", []):
                combined_text += " " + subtopic
        for bib in syllabus_data.get("bibliography", []):
            combined_text += " " + bib

        vectorizer = TfidfVectorizer(max_features=50, stop_words='english', ngram_range=(1, 2))
        try:
            tfidf_matrix = vectorizer.fit_transform([combined_text])
            feature_names = vectorizer.get_feature_names_out()
            return list(feature_names)
        except Exception as e:
            self.logger.warning(f"Error extrayendo terminología: {e}")
            return []

    def _calculate_terminology_usage(self, text: str, terminology: List[str]) -> float:
        """Calcula la densidad de uso de la terminología específica en el contenido."""
        content_lower = text.lower()
        term_count = sum(1 for term in terminology if term.lower() in content_lower)
        word_count = len(word_tokenize(text))
        term_density = (term_count * 1000) / word_count if word_count > 0 else 0
        # Supongamos que 5 términos por 1000 palabras es ideal
        return min(1.0, term_density / 5)

    def _calculate_subtopic_coverage(self, content: str, subtopics: List[str]) -> float:
        """Calcula la proporción de subtemas cubiertos en el contenido."""
        content_lower = content.lower()
        covered = 0
        for sub in subtopics:
            sub_terms = [w.lower() for w in word_tokenize(sub) if len(w) > 3]
            if sub_terms and sum(1 for term in sub_terms if term in content_lower) / len(sub_terms) >= 0.5:
                covered += 1
        return covered / len(subtopics) if subtopics else 0

    def _evaluate_lecture_notes(self, lecture_notes: str, topic_info: Dict[str, Any], course_terminology: List[str]) -> float:
        """Evalúa la calidad de las notas de clase combinando varias métricas."""
        scores = []
        # Cobertura de subtemas
        scores.append(self._calculate_subtopic_coverage(lecture_notes, topic_info.get("subtopics", [])))
        # Uso de terminología
        scores.append(self._calculate_terminology_usage(lecture_notes, course_terminology))
        # Legibilidad
        scores.append(self._calculate_readability(lecture_notes))
        # Estructura y organización
        scores.append(self._evaluate_content_structure(lecture_notes))
        # Presencia de ejemplos
        scores.append(self._evaluate_examples_presence(lecture_notes))
        return sum(scores) / len(scores)

    def _evaluate_practice_problems(self, practice_problems: str, topic_info: Dict[str, Any]) -> float:
        """Evalúa la calidad de los problemas de práctica."""
        scores = []
        scores.append(self._calculate_subtopic_coverage(practice_problems, topic_info.get("subtopics", [])))
        scores.append(self._evaluate_solutions_presence(practice_problems))
        scores.append(self._evaluate_difficulty_variety(practice_problems))
        scores.append(self._evaluate_problem_clarity(practice_problems))
        return sum(scores) / len(scores)

    def _evaluate_discussion_questions(self, discussion_questions: str, topic_info: Dict[str, Any]) -> float:
        """Evalúa la calidad de las preguntas para discusión."""
        scores = []
        scores.append(self._calculate_subtopic_coverage(discussion_questions, topic_info.get("subtopics", [])))
        scores.append(self._evaluate_critical_thinking(discussion_questions))
        scores.append(self._evaluate_open_ended_questions(discussion_questions))
        return sum(scores) / len(scores)

    def _evaluate_learning_objectives(self, learning_objectives: str, topic_info: Dict[str, Any]) -> float:
        """Evalúa la calidad de los objetivos de aprendizaje."""
        scores = []
        scores.append(self._calculate_subtopic_coverage(learning_objectives, topic_info.get("subtopics", [])))
        scores.append(self._evaluate_bloom_taxonomy_usage(learning_objectives))
        scores.append(self._evaluate_measurable_objectives(learning_objectives))
        return sum(scores) / len(scores)

    def _evaluate_suggested_resources(self, suggested_resources: str, topic_info: Dict[str, Any]) -> float:
        """Evalúa la calidad de los recursos sugeridos."""
        scores = []
        scores.append(self._calculate_subtopic_coverage(suggested_resources, topic_info.get("subtopics", [])))
        scores.append(self._evaluate_resource_variety(suggested_resources))
        scores.append(self._evaluate_resource_detail(suggested_resources))
        return sum(scores) / len(scores)
    
    # Métodos "stub" o simples para completar la evaluación
    def _evaluate_content_structure(self, text: str) -> float:
        paragraphs = [p for p in text.split("\n\n") if p.strip()]
        num_paragraphs = len(paragraphs)
        return 1.0 if num_paragraphs >= 5 else (num_paragraphs / 5.0 if num_paragraphs else 0.0)

    def _evaluate_examples_presence(self, text: str) -> float:
        return 1.0 if "ejemplo" in text.lower() else 0.0

    def _evaluate_solutions_presence(self, text: str) -> float:
        return 1.0 if "solución" in text.lower() or "solución:" in text.lower() else 0.0

    def _evaluate_difficulty_variety(self, text: str) -> float:
        # Este método puede analizar si se presentan problemas de distintos niveles; por ahora se devuelve un valor fijo
        return 0.8

    def _evaluate_problem_clarity(self, text: str) -> float:
        # Un ejemplo simple: si el texto tiene muchos signos de interrogación o puntos, se asume claridad
        return 0.8

    def _evaluate_critical_thinking(self, text: str) -> float:
        # Ejemplo simple: buscar palabras clave como "analiza", "discute", "reflexiona"
        keywords = ["analiza", "discute", "reflexiona", "argumenta"]
        count = sum(1 for kw in keywords if kw in text.lower())
        return min(1.0, count / len(keywords))

    def _evaluate_open_ended_questions(self, text: str) -> float:
        # Considerar preguntas que no tengan respuestas cerradas (por ejemplo, que terminen en "?")
        questions = [line for line in text.splitlines() if "?" in line]
        if not questions:
            return 0.0
        # Suponiendo que más del 50% de las preguntas son abiertas es bueno
        open_count = sum(1 for q in questions if not re.search(r'\b(?:sí|no)\b', q.lower()))
        return open_count / len(questions)

    def _evaluate_bloom_taxonomy_usage(self, text: str) -> float:
        # Buscar verbos de la taxonomía de Bloom (ej. "analiza", "aplica", "compara")
        bloom_verbs = ["analiza", "aplica", "compara", "evalúa", "crea", "sintetiza", "interpreta"]
        count = sum(1 for verb in bloom_verbs if verb in text.lower())
        return min(1.0, count / len(bloom_verbs))

    def _evaluate_measurable_objectives(self, text: str) -> float:
        # Verificar si se incluyen indicadores cuantificables o verbos medibles
        return 0.8 if any(kw in text.lower() for kw in ["porcentaje", "número", "cuantifica"]) else 0.5

    def _evaluate_resource_variety(self, text: str) -> float:
        # Ejemplo: contar la cantidad de recursos listados
        lines = [l for l in text.splitlines() if l.strip()]
        return min(1.0, len(lines) / 5.0)

    def _evaluate_resource_detail(self, text: str) -> float:
        # Evaluar si cada recurso viene con una breve descripción (esto es un ejemplo simple)
        return 0.8

    def _evaluate_consistency(self, generated_content: Dict[str, Dict[str, str]]) -> float:
        """
        Evalúa la consistencia semántica entre secciones usando TF-IDF y similitud de coseno.
        Se asume que mayor similitud entre secciones relacionadas (por ejemplo, entre 'lecture_notes'
        y 'learning_objectives') indica mayor consistencia.
        """
        texts = []
        for topic_content in generated_content.values():
            # Concatenar todos los textos de un tema
            combined = " ".join(topic_content.values())
            texts.append(combined)
        if len(texts) < 2:
            return 1.0  # Si solo hay un tema, no se puede evaluar consistencia entre temas
        
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf = vectorizer.fit_transform(texts)
        similarity_matrix = cosine_similarity(tfidf)
        # Excluir la diagonal y calcular el promedio de similitud
        n = similarity_matrix.shape[0]
        sum_sim = np.sum(similarity_matrix) - n  # restar la diagonal
        num_elements = n * (n - 1)
        avg_similarity = sum_sim / num_elements if num_elements > 0 else 1.0
        return avg_similarity


    
# evaluator = Evaluator(config)  # Puedes pasar una configuración si la tienes
# evaluacion = evaluator.evaluate_content(contenido, syllabus_data)
# print(evaluacion)


In [20]:

class InteractiveEducationalAgent:
    """Versión interactiva del agente educativo para Jupyter, simulando la carga de archivo."""
    def __init__(self):
        self.config = Config()
        self.file_path = None
        self.materials = None
        self.evaluation = None
        
        # Crear los componentes interactivos
        self._create_widgets()
        self._display_welcome()
    
    def _create_widgets(self):
        """Crear los widgets interactivos."""
        # Widget para cargar el archivo PDF (aunque no se usará para el procesamiento)
        self.upload_widget = widgets.FileUpload(
            description='Programa de curso:',
            accept='.pdf', 
            multiple=False
        )
        self.upload_widget.observe(self._handle_upload, names='value')
        
        # Botón para procesar el archivo
        self.process_button = widgets.Button(
            description='Procesar Programa',
            disabled=True,
            button_style='primary',
            icon='check'
        )
        self.process_button.on_click(self._on_process_click)
        
        # Salida de estado
        self.output = widgets.Output()
        
        # Barra de progreso
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=10,
            description='Progreso:',
            bar_style='info',
            orientation='horizontal'
        )
    
    def _display_welcome(self):
        """Mostrar la interfaz de bienvenida."""
        welcome_html = """
            <div style="background-color:#f8f9fa; padding:20px; border-radius:10px; margin-bottom:20px; color:#000000;">
                <h2 style="color:#0066cc;">🎓 Asistente de Generación de Materiales Educativos</h2>
                <p>Este asistente te ayudará a generar materiales educativos a partir del programa de un curso.</p>
                <p>Para comenzar, sube un archivo PDF, txt o .docx con el programa del curso utilizando el widget de carga.</p>
            </div>
        """
        display(HTML(welcome_html))
        display(widgets.VBox([self.upload_widget, self.process_button, self.progress, self.output]))
    
    def _handle_upload(self, change):
        """Manejar la carga del archivo."""
        if change['new']:
            # Se obtiene el primer archivo subido
            file_data = change['new'][0]  # Access the first element of the tuple
            filename = file_data['name']  # Access the filename directly
            content = file_data['content']  # Access the file content
            
            # Guardar el archivo localmente
            with open(filename, 'wb') as f:
                f.write(content)
            
            self.file_path = filename  # Se guarda el nombre, pero no se usará para el procesamiento real
            self.process_button.disabled = False
            
            with self.output:
                clear_output()
                print(f"✅ Archivo cargado: {filename}")
    
    def _on_process_click(self, b):
        """Manejar el clic del botón de procesamiento."""
        if not self.file_path:
            with self.output:
                clear_output()
                print("❌ Por favor, primero carga un archivo PDF.")
            return
        
        with self.output:
            clear_output()
            print("🚀 Iniciando procesamiento)...\n")
            
            # Inicializar componentes
            self.progress.value = 1
            print("Inicializando componentes...")
            llm_engine = LLMEngine(self.config)
            self.progress.value = 2
            generator = ContentGenerator(llm_engine, self.config)
            self.progress.value = 3
            processor = DocumentProcessor(self.config)
            self.progress.value = 4
            evaluator = Evaluator(self.config)
            
            # Procesar el archivo: se ignora el archivo cargado y se usa el fijo 'PROGRAMA_DE_CURSO.pdf'
            self.progress.value = 5
            syllabus_data =  processor.process_file(self.file_path)
            
            # Generar materiales
            self.progress.value = 6
            print("\n📝 Generando materiales educativos...")
            self.materials = generator.generate_all_materials(syllabus_data)
            
            # Evaluar contenido
            self.progress.value = 8
            print("\n🔍 Evaluando calidad del contenido...")
            self.evaluation = evaluator.evaluate_content(self.materials, syllabus_data)
            
            # Generar PDF
            self.progress.value = 9
            print("\n📄 Generando documento PDF...")
            pdf_success = dict_to_pdf(self.materials, "materiales_curso.pdf")
            
            # Finalizar
            self.progress.value = 10
            
            print("\n✅ Proceso completado con éxito!")
                # display(FileLink('materiales_curso.pdf'))
                
                # Mostrar resumen de evaluación
            evaluation_html = """
                <div style="background-color:#f0f7ff; padding:15px; border-radius:5px; margin-top:20px;">
                    <h3>📊 Resumen de evaluación</h3>
                    <table style="width:100%; border-collapse: collapse;" border="1">
                        <tr><th>Tipo de Contenido</th><th>Puntuación</th></tr>
                """

                # Agregar filas de content_type_scores
            for key, value in self.evaluation["content_type_scores"].items():
                    evaluation_html += f"""
                    <tr>
                        <td>{key.replace("_", " ").title()}</td>
                        <td>{value * 100:.0f}%</td>
                    </tr>
                    """

                # Agregar métricas generales
            evaluation_html += """
                    <tr><th colspan="2">Métricas Generales</th></tr>
                """

            for key, value in  self.evaluation["overall_metrics"].items():
                    evaluation_html += f"""
                    <tr>
                        <td>{key.replace("_", " ").title()}</td>
                        <td>{value * 100:.0f}%</td>
                    </tr>
                    """

                # Agregar promedio general
            evaluation_html += f"""
                    <tr>
                        <td><b>Puntuación Promedio</b></td>
                        <td><b>{ self.evaluation["average_score"] * 100:.0f}%</b></td>
                    </tr>
                """

            evaluation_html += "</table></div>"

                # Mostrar HTML en Jupyter Notebook
            display(HTML(evaluation_html))


# Crear y mostrar el agente inte
agent = InteractiveEducationalAgent()
